# 0. Instalar e importar bibliotecas 📚

In [10]:
# !pip install requests
# !pip install requests_oauthlib
# !pip install oauthlib
# !pip install pandas

In [11]:
import requests
from requests_oauthlib import OAuth2Session
from requests_oauthlib import OAuth2Session
from requests.auth import HTTPBasicAuth
from oauthlib.oauth2 import BackendApplicationClient
import json
from tqdm import tqdm

In [3]:
from datetime import datetime as dt
import pandas as pd

In [4]:
from rjson import RenderJSON #funcionalidade para colapsar as saídas em json

In [5]:
import os

# 1. Obter token de acesso 🔑

Usar o arquivo json gerado no notebook credentials com client_id e client_secret.

In [6]:
keys = json.loads(open('keys_prod.json', 'r').read())

Ou preencher manualmente os dados das credenciais de acesso

In [7]:
# keys = {
#     "client_id": "PREENCHER COM CLIENT ID",
#     "client_secret": "PREENCHER COM CLIENT SECRET",
# }

In [8]:
client_id = keys['client_id']
client_secret = keys['client_secret']
scope= "accountability.statements accountability.request"
# scope= "accountability.request"

In [9]:
def get_token(client_id, client_secret):
    auth = HTTPBasicAuth(client_id, client_secret)
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client) 
    oauth = OAuth2Session(client=client, scope=scope)
    return oauth.fetch_token(token_url='https://oauth.bb.com.br/oauth/token', auth=auth)
token = get_token(keys['client_id'], keys['client_secret'])

In [12]:
print('validade do token: ', dt.strftime(dt.fromtimestamp(token['expires_at']), '%d/%m/%y %H:%M:%S'))

validade do token:  22/11/24 19:11:52


# 2. Requisições 📞

Criar cabeçalho com a chave da aplicação de homologação e token de acesso.

In [13]:
gw_dev_app_key = keys['gw_dev_app_key']

In [14]:
def set_headers():
    gw_dev_app_key = keys['gw_dev_app_key']
    headers = {
        #     'accept': 'application/json',
            'Authorization': 'Bearer '+token['access_token'],
            'X-Application-Key' : gw_dev_app_key
            }
    return headers

In [15]:
path = 'https://api.bb.com.br/accountability/v3'

## API 📕

In [16]:
token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()
print(f"{path}/swagger")
r = requests.get(f"{path}/swagger", headers=headers)
# r.json()

https://api.bb.com.br/accountability/v3/swagger


In [17]:
with open('api_prod.json','w') as f:
    f.write(str(r.json()))
    f.close() 

In [18]:
# r.json()
RenderJSON(r.json())

# Contas autorizadas

In [ ]:
headers = set_headers()

url = f"{path}/conta-corrente/orgaos-controle?gw-dev-app-key={gw_dev_app_key}&numeroRegistro=100"
print(url)
r = requests.get(url, headers=headers)
r.json() #RenderJSON(r.json())

In [26]:
headers = set_headers()

for i in range(0,300,100):
    url = f"{path}/conta-corrente/orgaos-controle?gw-dev-app-key={gw_dev_app_key}&numeroRegistro={i}"
    # url = f"{path}/conta-corrente/orgaos-controle?gw-dev-app-key={gw_dev_app_key}"
    print(url)
    r = requests.get(url, headers=headers)
    print(r.json())

https://api.bb.com.br/accountability/v3/conta-corrente/orgaos-controle?gw-dev-app-key=ad87f4e00cbd95ca2cff2a6be628877f&numeroRegistro=0
{'numeroRegistroConsultar': 56310, 'quantidadeContaCorrente': 100, 'listaContaCorrente': [{'codigoProgramaGoverno': 159, 'nomeProgramaGoverno': 'CONTAS PR PRIAS', 'cnpj': 1614878000180, 'agencia': 7, 'nomeAgencia': 'RECIFE', 'numeroContaCorrente': 5636}, {'codigoProgramaGoverno': 159, 'nomeProgramaGoverno': 'CONTAS PR PRIAS', 'cnpj': 10565000000192, 'agencia': 7, 'nomeAgencia': 'RECIFE', 'numeroContaCorrente': 5672}, {'codigoProgramaGoverno': 159, 'nomeProgramaGoverno': 'CONTAS PR PRIAS', 'cnpj': 10572071004533, 'agencia': 7, 'nomeAgencia': 'RECIFE', 'numeroContaCorrente': 16017}, {'codigoProgramaGoverno': 159, 'nomeProgramaGoverno': 'CONTAS PR PRIAS', 'cnpj': 10572071029285, 'agencia': 7, 'nomeAgencia': 'RECIFE', 'numeroContaCorrente': 16150}, {'codigoProgramaGoverno': 1, 'nomeProgramaGoverno': 'PNAE', 'cnpj': 10565000000192, 'agencia': 7, 'nomeAgenci

In [27]:
contas = pd.json_normalize(r.json()['listaContaCorrente'])

In [28]:
contas

,codigoProgramaGoverno,nomeProgramaGoverno,cnpj,agencia,nomeAgencia,numeroContaCorrente
0,159,CONTAS PR PRIAS,1614878000180,7,RECIFE,5636
1,159,CONTAS PR PRIAS,10565000000192,7,RECIFE,5672
2,159,CONTAS PR PRIAS,10572071004533,7,RECIFE,16017
3,159,CONTAS PR PRIAS,10572071029285,7,RECIFE,16150
4,1,PNAE,10565000000192,7,RECIFE,28900
...,...,...,...,...,...,...
95,25,BPC ESCOLA,12057383000113,67,GARANHUNS,47169
96,19,AEPETI,10782874000100,67,GARANHUNS,47170
97,19,AEPETI,12057383000113,67,GARANHUNS,47171
98,21,IGD BOLSA FAMÍLIA,12057383000113,67,GARANHUNS,47172


In [29]:
contas.to_csv('../data/contas.csv')

## Agencias

In [ ]:
cnpj = '11435633000149'
cep = '50050910'

# token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

print(f"{path}/agencias-proximas?cnpj={cnpj}&cep={cep}")
r = requests.get(f"{path}/agencias-proximas?cnpj={cnpj}&cep={cep}", headers=headers)
r.json() #RenderJSON(r.json())

## Programas

### Progamas-categorias

In [ ]:
numeroProgramaGoverno = "1"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/programas-governo/{numeroProgramaGoverno}/categorias", headers=headers)

r.json() #RenderJSON(r.json())

### Progamas-lancamentos

In [ ]:
numeroProgramaGoverno = "3"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    'dataInicio': '2018-10-01',
    'dataFim': '2018-10-30',
    'pagina': 1
}

r = requests.get(f"{path}/programas-governo/{numeroProgramaGoverno}/orgaos-repasse/lancamentos-atualizados", headers=headers, params=body)

r.json() #RenderJSON(r.json())

### Progamas-sublancamentos

In [ ]:
numeroProgramaGoverno = "3"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    'dataInicio': '2018-10-01',
    'dataFim': '2018-10-30',
    'pagina': 1
}

r = requests.get(f"{path}/programas-governo/{numeroProgramaGoverno}/orgaos-repasse/sublancamentos-atualizados", headers=headers, params=body)

r.json() #RenderJSON(r.json())

## Extratos

### Extratos-transacoes

In [37]:
branchCode = contas.iloc[0]['agencia']
accountNumber = contas.iloc[0]['numeroContaCorrente']
data_inicio = "2024-01-01"
data_fim = "2024-01-30"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

url = f"{path}/statements/{branchCode}-{accountNumber}/control-agencies?startDate={data_inicio}&endDate={data_fim}"
print(url)

r = requests.get(url, headers=headers)
r.json() #RenderJSON(r.json())

https://api.bb.com.br/accountability/v3/statements/7-5636/control-agencies?startDate=2024-01-01&endDate=2024-01-30


{'error': 'Não existem lançamentos para a conta no período informado'}

### Extratos-subtransacoes

In [39]:
branchCode = "1"
accountNumber = "2"
id = 6

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/statements/{branchCode}-{accountNumber}/debits/{id}/subtransactions", headers=headers)
r.json() #RenderJSON(r.json())

{'error': 'Agência e Conta não está vinculado ao Órgão Repassador.'}

## Despesas

### Despesas-transacao-documentos

In [ ]:
branchCode = "1"
accountNumber = "2"
transactionId = "7"
documentId = "26"
bookingDate = "2018-10-11"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/expenses/{branchCode}-{accountNumber}/transactions/{transactionId}/documents/{documentId}?bookingDate={bookingDate}", headers=headers)
r.json() #RenderJSON(r.json())

### Despesas-subtransacao-documentos

In [ ]:
branchCode = "1"
accountNumber = "2"
transactionId = "6"
subTransactionId = "1"
documentId = "106"
bookingDate = "2018-10-11"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/expenses/{branchCode}-{accountNumber}/transactions/{transactionId}/subTransactions/{subTransactionId}/documents/{documentId}?bookingDate={bookingDate}", headers=headers)
r.json() #RenderJSON(r.json())

## Aplicacoes

### Aplicacoes-fundos

In [43]:
agencia = "1"
contaCorrente = "2"
fundosInvestimentoId = '1'


token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    "mes": 10,
    "ano": 2018
    }

r = requests.get(f"{path}/extratos/{agencia}-{contaCorrente}/fundos-investimentos/{fundosInvestimentoId}", headers=headers, params=body)
r.json() #RenderJSON(r.json())

{'errors': [{'code': '3164477.1',
   'message': 'Agência e Conta não está vinculado ao Órgão Repassador.'}]}

In [44]:
agencia = "1"
contaCorrente = "2"
fundosInvestimentoId = '1'

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    "mes": 10,
    "ano": 2018
    }

r = requests.get(f"{path}/extratos/{agencia}-{contaCorrente}/fundos-investimentos/{fundosInvestimentoId}/control-agencies", headers=headers, params= body)
r.json() #RenderJSON(r.json())

{'chaveCorrelacao': 'DfTd4XX2qbsOAs0sLA7s0101',
 'codigoErro': '5',
 'codigoMensagem': 999999999,
 'localizedMessage': 'ERRO GET CONTAINER PRM-ERRO, RESP=00000022, RESP-2=00000011 - *** GFAS9023 *** (M005-002)',
 'message': 'ERRO GET CONTAINER PRM-ERRO, RESP=00000022, RESP-2=00000011 - *** GFAS9023 *** (M005-002)',
 'numeroOcorrencia': '004983934512',
 'prefixoCodigoErro': 'M',
 'providencia': '',
 'sequencialErro': '2',
 'sequencialEvento': 2,
 'textoErro': 'ERRO GET CONTAINER PRM-ERRO, RESP=00000022, RESP-2=00000011 - *** GFAS9023 ***',
 'versaoMensagem': 1}

### Aplicacoes-poupanca

In [ ]:
agencia = "1"
contaCorrente = "2"
variacaoPoupanca = '013'

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    "mes": 10,
    "ano": 2018
    }

r = requests.get(f"{path}/extratos/{agencia}-{contaCorrente}/poupanca/{variacaoPoupanca}", headers=headers, params=body)
r.json() #RenderJSON(r.json())

In [ ]:
agencia = "1"
contaCorrente = "2"
variacaoPoupanca = '013'

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    "mes": 10,
    "ano": 2018
    }

r = requests.get(f"{path}/extratos/{agencia}-{contaCorrente}/poupanca/{variacaoPoupanca}/orgao-controle", headers=headers, params=body)
r.json() #RenderJSON(r.json())

## Saldos

### Conta Corrente

In [ ]:
agencia = "1"
contaCorrente = "2"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/saldos/{agencia}-{contaCorrente}/conta-corrente", headers=headers)
r.json() #RenderJSON(r.json())

### Aplicações financeiras

In [ ]:
agencia = "1"
contaCorrente = "2"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/saldos/{agencia}-{contaCorrente}/aplicacoes-financeiras", headers=headers)
r.json() #RenderJSON(r.json())

## Orgaos

### Débito

In [ ]:
agencia = "1"
contaCorrente = "2"
data_inicio = "2018-10-01"
data_fim = "2018-10-30"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    'numeroPagina': 1
}

r = requests.get(f"{path}/orgaos-repasse/{agencia}-{contaCorrente}/lancamentos-debito", headers=headers, params=body)
r.json()

In [ ]:
agencia = "1"
contaCorrente = "2"
data_inicio = "2018-10-01"
data_fim = "2018-10-30"
ordemBancaria = "1"
item = "1"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

r = requests.get(f"{path}/orgaos-repasse/{agencia}-{contaCorrente}/lancamentos-debito/{ordemBancaria}-{item}/categorias-despesa", headers=headers)
r.json()

### Crébito

In [ ]:
agencia = "1"
contaCorrente = "2"
data_inicio = "2018-10-01"
data_fim = "2018-10-30"

token = get_token(keys['client_id'], keys['client_secret'])
headers = set_headers()

body = {
    'numeroPagina': 1
}

r = requests.get(f"{path}/orgaos-repasse/{agencia}-{contaCorrente}/lancamentos-credito", headers=headers, params=body)
r.json()
